In [4]:
# from pyDWSIMconnect import PyDWSIMconnect
from simulation import Simulation
import numpy as np
import time
from fobj import *
from scipy import optimize
from pprint import pprint

In [2]:
# Loading DWSIM simulation into Python (Simulation object)
sim_smr = Simulation(path2sim="C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep.dwxmz")
sim_smr.Add_refs()
sim_smr.Connect()

added refs
Simulation was loaded successfully


In [24]:
def fobj8n_2exp(sim_smr, x):
    mr1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("MR-1")
    comp1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-1") 
    comp2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-2") 
    comp3 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-3") 
    comp4 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-4") 
    pump1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01") 
    pump2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-02") 
    vlv1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("VALV-01")
    cool8 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-08")
    lng = sim_smr.flowsheet.GetFlowsheetSimulationObject("LNG-1")
    if sim_smr.x is None:
        sim_smr.x = np.zeros(len(x))
    if np.linalg.norm(sim_smr.x - np.asarray(x))>1e-10:
    # m_old = np.ones(mr1.Phases[0].Compounds.Values.Count)
    # ite=0
    # for m_i in mr1.Phases[0].Compounds.Values:
    #     m_old[ite] = m_i.MassFraction*mr1.GetMassFlow()
    #     ite+=1
    # x_old = np.array( [m_old[7],m_old[0],m_old[1],m_old[2],m_old[5], vlv1.OutletPressure, comp4.POut, cool8.OutletTemperature] ) 
    # if np.linalg.norm( x-x_old ) > 1e-5:
    # if ( abs(mr1.GetMassFlow() - x[0])/abs(x[0]) + abs(vlv1.OutletPressure - x[1])/abs(x[1]) + abs(comp4.POut - x[2])/abs(x[2]) ) > 1e-12:
        # print("calculated with ")
        # print( abs(mr1.GetMassFlow() - x[0])/abs(x[0]))
        # print(abs(vlv1.OutletPressure - x[1])/abs(x[1]))
        # print(abs(comp4.POut - x[2])/abs(x[2]) )
        # mr1.SetMassFlow(x[0])
        sep1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("SEP-02")
        sep2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("SEP-03")
        sep1.GraphicObject.Activate = False
        sep2.GraphicObject.Activate = False

        mr1.SetOverallCompoundMassFlow(0,x[1])
        mr1.SetOverallCompoundMassFlow(1,x[2])
        mr1.SetOverallCompoundMassFlow(2,x[3])
        mr1.SetOverallCompoundMassFlow(5,x[4])
        mr1.SetOverallCompoundMassFlow(7,x[0])
        vlv1.OutletPressure = x[5]
        comp4.POut = x[6]
        cool8.OutletTemperature = x[7]

        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-03").Phases[1].Properties.massfraction < 1e-5:
            pump1.GraphicObject.Active = False
            print(sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01").GraphicObject.Active)
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
        else:
            pump1.GraphicObject.Active = True
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject) #avoid bug
            sim_smr.flowsheet.ConnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject,
                -1,-1)
        sep1.GraphicObject.Active = True
        sep1.Calculate()
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-05").Phases[1].Properties.massfraction < 1e-5:
            pump2.GraphicObject.Active = False
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
        else:
            pump1.GraphicObject.Active = True
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
            sim_smr.flowsheet.ConnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject,
                -1,-1)
        sep2.GraphicObject.Active = True
        sep2.Calculate()
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        #force to calculate spec2 spec3 cool6 e cool5
        sim_smr.flowsheet.GetFlowsheetSimulationObject("SPEC-02").Solve()
        sim_smr.flowsheet.GetFlowsheetSimulationObject("SPEC-03").Solve()
        sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-06").Solve()
        sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-05").Solve()
        time.sleep(0.05)
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        time.sleep(0.05)
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        time.sleep(0.05)
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        sumW = (comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-03").Phases[1].Properties.massfraction > 1e-5:
            sumW += pump1.DeltaQ 
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-05").Phases[1].Properties.massfraction > 1e-5:    
            sumW += pump2.DeltaQ
        sumW = sumW/lng.GetMassFlow()/3600
        mita1 = min(sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-08").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-17").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-09").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-17").GetTemperature(), 
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-1_2").GetTemperature()   - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-17").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-10").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-15").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-11").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-15").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-2").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-15").GetTemperature()
        )
        mita2 = min(sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-10").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-16").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-2").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-16").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-12").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-13").GetTemperature(),
            sim_smr.flowsheet.GetFlowsheetSimulationObject("NG-3").GetTemperature() - sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-13").GetTemperature()
        )
    else:
        sumW = sim_smr.f
        mita1 = sim_smr.g
        mita2 = sim_smr.g
        # print("skipped with")
        # print( abs(mr1.GetMassFlow() - x[0])/abs(x[0]))
        # print(abs(vlv1.OutletPressure - x[1])/abs(x[1]))
        # print(abs(comp4.POut - x[2])/abs(x[2]) )
    # if np.linalg.norm( x-x_old ) > 1e-5:
    #     sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.path,True)
    sim_smr.x = x
    sim_smr.f = sumW
    sim_smr.g = min(mita1, mita2)
    return sumW, min(mita1, mita2)

In [43]:
# Initial simulation setup
x0 = np.array( [0.25/3600, 0.70/3600, 1.0/3600, 1.10/3600, 1.80/3600, 2.50e5, 50.00e5, -40+273.15] )
sim_smr.fobj = lambda x: fobj8n_2exp(sim_smr, x)
sumW, mita = sim_smr.fobj(0.95*x0)
print(sim_smr.x,
sim_smr.f,
sim_smr.g)
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,
        'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep2.dwxmz',True)

TypeError: '<' not supported between instances of 'NoneType' and 'float'

In [29]:
print(sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01").GraphicObject.Active)

True


In [20]:
x_top = np.array( [0.252779/3600, 0.441063/3600, 1.411068/3600, 0.805952/3600, 1.851257/3600, 3.342832e5, 34.97221e5, -27.2958+273.15] )
args=(sim_smr)
xtol=0.01
ftol=0.01
maxiter=50 # +- 5 seconds per iteration
print("starting optimization")
iteration = 0
regularizer = np.array([1e4,1e4,1e4,1e4,1e4, 1e-5, 1e-6, 1e-2])
#rever \/
bounds_raw = np.array( [0.6*np.asarray(x0), 1.4*np.asarray(x0)] )
bounds_reg = regularizer*bounds_raw
bounds_reg[0][-1] = 153
bounds_reg[1][-1] = 233
print(bounds_reg)
bounds = optimize.Bounds(bounds_reg[0], bounds_reg[1])
g = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[1]
nonlinear_constraint = optimize.NonlinearConstraint(g, 3, np.inf, jac='2-point', hess=optimize.BFGS())
f = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[0]
# print(fobj8n_2exp(sim_smr,x_top))

starting optimization
[[  0.41666667   1.16666667   1.66666667   1.83333333   3.
    1.5          3.         153.        ]
 [  0.97222222   2.72222222   3.88888889   4.27777778   7.
    3.5          7.         233.        ]]


In [22]:
fobj8n_2exp(sim_smr, x_top)
pprint(vars(sim_smr))

{'f': 0.2878099866914784,
 'flowsheet': <DWSIM.UI.Desktop.Shared.Flowsheet object at 0x0000020EDB65A9D0>,
 'fobj': <function <lambda> at 0x0000020EE2CD5820>,
 'g': 2.103050383381003,
 'interface': <DWSIM.Automation.Automation2 object at 0x0000020EDB654EB0>,
 'path': 'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep.dwxmz',
 'path2dwsim': 'C:\\Users\\lfsfr\\AppData\\Local\\DWSIM7\\',
 'x': array([7.02163889e-05, 1.22517500e-04, 3.91963333e-04, 2.23875556e-04,
       5.14238056e-04, 3.34283200e+05, 3.49722100e+06, 2.45854200e+02])}


In [46]:
# Local optimization with trust-region -> working to some extent
result = optimize.minimize( f, np.asarray(x0)*regularizer, 
            method='trust-constr', jac='2-point', hess=optimize.BFGS(),
            constraints=[nonlinear_constraint], bounds=bounds, callback=None,
            options={'verbose': 3, 
                     'xtol': xtol, 
                     'maxiter': 0.1*maxiter, 
                     'finite_diff_rel_step': None, 
                     'initial_tr_radius': 0.1} )

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |   9   |   0   | +3.9330e-01 | 1.00e-01 | 5.98e-02 | 1.51e+02 | 1.00e+00 |  1.00e-01   |   0   |


c:\Users\lfsfr\Desktop\pyDWSIMopt\venv\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


|   2   |  18   |   1   | +3.9167e-01 | 7.00e-01 | 6.01e-02 | 1.51e+02 | 1.00e+00 |  1.00e-01   |   2   |
|   3   |  27   |   2   | +3.8657e-01 | 4.41e+00 | 3.56e-02 | 1.50e+02 | 1.00e+00 |  1.00e-01   |   4   |
|   4   |  36   |   5   | +3.8657e-01 | 1.33e+00 | 3.56e-02 | 1.50e+02 | 1.00e+00 |  1.00e-01   |   2   |
|   5   |  45   |   8   | +3.8860e-01 | 1.33e+00 | 3.11e-02 | 1.51e+02 | 1.00e+00 |  1.00e-01   |   2   |

The maximum number of function evaluations is exceeded.
Number of iterations: 5, function evaluations: 45, CG iterations: 8, optimality: 3.11e-02, constraint violation: 1.51e+02, execution time: 2.8e+01 s.


In [47]:
print(f(result.x))
print(g(result.x))
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,
        'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep2.dwxmz',True)
print(result.x)
# x_top
bounds


0.38860004298120365
4.020925209364293
[0.69444444 1.94444444 2.77777778 3.05555556 5.         2.56513715
 4.9774209  2.44793885]


Bounds(array([  0.41666667,   1.16666667,   1.66666667,   1.83333333,
         3.        ,   1.5       ,   3.        , 153.        ]), array([  0.97222222,   2.72222222,   3.88888889,   4.27777778,
         7.        ,   3.5       ,   7.        , 233.        ]))

In [48]:
# Global optimization with Differential Evolution
result = optimize.differential_evolution(f, bounds, constraints=[nonlinear_constraint], 
            maxiter=5, popsize=5, seed=1234, disp=True, polish=False)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
differential_evolution step 1: f(x)= 0.263149
False
False
False
False
False
False
False
differential_evolution step 2: f(x)= 0.263149
False
False
False
False
False
False
False
False
False
False
False
differential_evolution step 3: f(x)= 0.263149
